<a href="https://colab.research.google.com/github/aferron/Dream/blob/main/EnDex_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

As a demo, this notebook will be set up with a runnable version of the EnDex evaluation model. The model data will be uploaded into the shared drive so that it can be accessed here. 

Connecting to the drive folder:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Requirements block:

In [ ]:
!python --version

!pip install torch==1.10.1 torchvision==0.11.2 torchaudio==0.10.1
!pip install transformers==4.14.0

The sample code provided by the EnDex team to run the model:

In [55]:
metric_running: str = "EnDexAsReleased"
directory_location: str = "/content/drive/Shareddrives/DRAEM/"
dataset_location: str = "Reflect/data/"
dataset_name: str = "Reflect"
results_dir: str = "results/"
name_of_dataset: str = "organized_Reflect_9k_responses_test"
model_name: str = "endex_model"
tokenizer_model: str = "roberta-large"
results_prefix: str = "results_summary_for_"
json_ext: str = '.json'
csv_ext: str = '.csv'
results: str = 'results'
verbose: bool = True
run_example: bool = False

In [ ]:
import torch
from transformers import RobertaTokenizer,RobertaForSequenceClassification
tokenizer = RobertaTokenizer.from_pretrained(tokenizer_model)
model_dir = directory_location + model_name
model = RobertaForSequenceClassification.from_pretrained(model_dir, local_files_only=True)

In [ ]:
if run_example:
  inputs = tokenizer("it's such a great point, and i'd love to hear back on your thoughts!", return_tensors="pt")
  with torch.no_grad():
      logits = model(**inputs).logits
  predicted_class_id = logits.argmax().item()
  print('the engagingness prediction of the input sentence is: ', predicted_class_id)

In [58]:
import datetime
import json
import os
from tqdm import tqdm

class evaluate_data_set:
  def __init__(self, model) -> None:
    self.data_file = open(directory_location  + dataset_location + name_of_dataset + json_ext)
    self.reflect_data = json.load(self.data_file)
    self.data_file.close()
    self.engaging_responses = 0
    self.total = 0
    self.total_per_dimension = [0 for i in range(6)]
    self.engaging_per_dimension = [0 for i in range(6)]
    now = datetime.datetime.now()
    self.time_salt = str(now.year) + str(now.month) + str(now.day) + "_" + str(now.hour) + str(now.minute)
    self.num_engaging = "number engaging:"
    self.total_eval = "total evaluated:"
    self.per_engaging = "percent engaging:"
    self.file_path_error_msg = "The results file path doesn't exist"
    self.file_error_msg = "The results file already exists"

  def evaluate_response(self, response: str, dimension: int) -> int:
    inputs = tokenizer(response, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_class_id = logits.argmax().item()
    self.engaging_responses += predicted_class_id
    self.total += 1
    self.total_per_dimension[dimension] += 1
    self.engaging_per_dimension[dimension] += predicted_class_id
    return predicted_class_id

  def get_result_string(self):
    percent_engaging = self.engaging_responses/self.total
    percent_engaging_per_dim = [0 for i in range(6)]
    for i in range (1, 6):
      percent_engaging_per_dim[i] = self.engaging_per_dimension[i] / self.total_per_dimension[i]

    summary = self.num_engaging + str(self.engaging_responses) + '\n' + self.total_eval + str(self.total) + '\n' + self.per_engaging + str(percent_engaging)

    csv = ",total,dim 1,dim 2,dim 3,dim 4,dim 5\n" + self.num_engaging + "," + str(self.engaging_responses)

    for i in range(1, 6):
      csv += "," + str(self.engaging_per_dimension[i])

    csv += '\n' + self.total_eval + "," + str(self.total)

    for i in range(1, 6):
      csv += ',' + str(self.total_per_dimension[i])

    csv += '\n' + self.per_engaging + "," + str(percent_engaging)

    for i in range (1, 6):
      csv += ',' + str(percent_engaging_per_dim[i])
      percent_engaging_per_dim[i] = self.engaging_per_dimension[i] / self.total_per_dimension[i]
      summary += "\ntotal in dimension {dim}: ".format(dim = i) + str(self.total_per_dimension[i])
      summary += "\nengaging in dimension {dim}: ".format(dim = i) + str(self.engaging_per_dimension[i])
      summary += "\npercent engaging in dimension {dim}: ".format(dim = i) + str(percent_engaging_per_dim[i])

    summary += '\n\n\n'

    return (summary, csv)

  def evaluate_all_reflect_responses(self) -> float:
    results_file_path = directory_location + results_dir
    results_file_name = results_prefix + metric_running + '_on_' + dataset_name + '_' + self.time_salt + csv_ext
    new_data_file =  directory_location  + dataset_location + name_of_dataset + '_' + metric_running + '_' + results + '_' + self.time_salt + json_ext
    if not os.path.exists(results_file_path):
      print(self.file_path_error_msg)
      exit()
    if os.path.isfile(results_file_name):
      print(self.file_error_msg)
      exit()
    for i in tqdm(self.reflect_data):
      for num in range(1, 6):
          for j, response in enumerate(i['responses_{num}'.format(num = num)]):
            engagingness = self.evaluate_response(response, num)
            response = [response, {metric_running: engagingness}]
            i['responses_{num}'.format(num = num)][j] = response
    if verbose:
      print(self.reflect_data[0])
    self.data_file = open(new_data_file, 'w')
    self.data_file.write(json.dumps(self.reflect_data))
    self.data_file.close()
    results_summary = self.get_result_string()
    if verbose:
      print(results_summary[0])
      print(results_summary[1])
    results_file = open(results_file_path + results_file_name, 'x')
    results_file.write(results_summary[1])
    results_file.close()

In [ ]:
evaluate_data_set(model).evaluate_all_reflect_responses()